In [26]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import datetime as dt


# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('quickstart-1581730772240-cc225e029ff1.json', scope)
gc = gspread.authorize(creds)

# Find a workbook by name and open the 'Q3 data' sheet
# Make sure you use the right name here.
workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1dYKJFZkZ-qTcQOrbf80u85Zl1ugrRly3p_AJaQixgcw/edit#gid=1910446069')
w1 = workbook.worksheet('Q2 data')
w2 = workbook.worksheet('Q3 data')


# Extract and print all of the values and load data
list_of_hashes = w2.get_all_records()
df = pd.DataFrame.from_dict(list_of_hashes,orient='columns')
df.head()

,ts,user_id,country_id,site_id
0,2019-02-01 00:01:24,LC36FC,TL6,N0OTG
1,2019-02-01 00:10:19,LC39B6,TL6,N0OTG
2,2019-02-01 00:21:50,LC3500,TL6,N0OTG
3,2019-02-01 00:22:50,LC374F,TL6,N0OTG
4,2019-02-01 00:23:44,LCC1C3,TL6,QGO3G


#### Question1

In [157]:
#filter by country_id = 'BDV'
df_bdv = df[df.country_id=='BDV']   
#group by site_id and count unique user_id
df_bdv_group = df_bdv.groupby('site_id')['user_id'].nunique().to_frame()
df_bdv_group.rename(columns={'user_id':'count'}, inplace=True)
#sort by unique user number and then take the top one
df_bdv_group.sort_values('count',ascending=False).head(1)

,count
site_id,
5NPAU,544


The rows where the 'country_id'='BDV' were selected first(df_bdv). Then, the number of unique 'user_id' was computed by 'site_id'. As shown above, site, 5NPAU has the largest number of unique users, 544.   

#### Question2 

In [54]:
#convert ts to timestamp
df.ts = pd.to_datetime(df.ts)
begin = dt.datetime(2019,2,3,0,0,0)      # 2019-02-03 00:00:00
end = dt.datetime(2019,2,4,23,59,59)     # 2019-02-04 23:59:59

#filter by time
df_time = df.loc[(df.ts>begin) & (df.ts<end)]

#for each user_id and site_id, count visited times
df_count = df_time.groupby(['user_id','site_id'])['ts'].count().to_frame().reset_index()
df_count.rename(columns={'ts':'counts'}, inplace=True)

#filter by visit times greater than 10
df_count[df_count.counts>10]

,user_id,site_id,counts
3,LC06C3,N0OTG,25
417,LC3A59,N0OTG,26
485,LC3C7E,3POLC,15
493,LC3C9D,N0OTG,17


First, 'ts' was converted to timestamp and the dataset was filtered by the times that were given. From the subset of the full data, for each user_id and site_id, visited times were counted. As shown above, there are 4 users who visited a certain site more than 10 times. 

#### Question3

In [72]:
#for each user_id, get last visit time
df_last_visit = df.groupby('user_id')['ts'].max().to_frame().reset_index()

#merge with full table to get the last visit site
df_last_site = df_last_visit.merge(df, on=['ts','user_id'], how='left')

#change column names
df_last_site.columns = ['user_id','ts_last','country_last','site_last']

#group by last visit site and count each site's occurrence, sort by counts
df_user_count=df_last_site.groupby('site_last')['ts_last'].count().to_frame().sort_values('ts_last',ascending=False)
df_user_count.rename(columns={'ts_last':'num_users'}, inplace=True)
df_user_count

,num_users
site_last,
5NPAU,992
N0OTG,561
QGO3G,289
GVOFK,42
3POLC,28
RT9Z6,2
EUZ/Q,1
JSUUP,1


By using max() function on 'ts', I could get the list of last visit sites from this data. For each site, the unique number of users were counted. As shown above, site "3POLC" is ranked at 5th with 28 users whose last visit in the dataset was to "3POLC". The top three sites are "5NPAU", "N0OTG", and "QGO3G" 

#### Question4

In [75]:
#for each user, get first visit time
df_first_visit = df.groupby('user_id')['ts'].min().to_frame().reset_index()

#merge with full table to get the first visit site
df_first_site = df_first_visit.merge(df, on=['ts','user_id'], how='left')

#change column names
df_first_site.columns = ['user_id','ts_first','country_first','site_first']

#combine with the last visit table from Question3, join by using user_id
df_comb = df_last_site.merge(df_first_site, on='user_id', how='left')

#filter by last visit site=first visit site and compute the length
len(df_comb[df_comb.site_first==df_comb.site_last])

1670

By using min() function on 'ts', I could get the list of the first visit sites. Then this dataset was merged with the dataset with the last visit sites based on their 'user_id'. And the number of users whose first and last visits are the same were computed. 1670 users have the same first and last visit sites.

#### Question5

In [155]:
df_id = df[['user_id','country_id']].drop_duplicates()
df_id = df_id.groupby('user_id')['country_id'].count().to_frame()
df_id = df_id[df_id.country_id>1]
df_id = df_id.drop('country_id', axis=1)
df_id

#In order to get A 
df_a = df[['user_id','site_id']].drop_duplicates()
df_a = df_id.merge(df_a, on=['user_id'], how='left')
df_a = df_a.groupby('site_id')['user_id'].count().to_frame().reset_index()
df_a.rename(columns={'user_id':'A'}, inplace=True)

#In order to get B
df_b = df[['user_id','site_id']].drop_duplicates()
df_b = df_b.groupby('site_id')['user_id'].count().to_frame().reset_index()
df_b.rename(columns={'user_id':'B'}, inplace=True)

#merge A and B
df_ab = df_a.merge(df_b, on=['site_id'], how='left')
df_ab['ratio'] = round(df_ab['B']/df_ab['A'], 2)
df_ab.sort_values('ratio',ascending=False)

,site_id,A,B,ratio
6,QGO3G,79,353,4.47
1,5NPAU,260,1104,4.25
5,N0OTG,179,658,3.68
0,3POLC,15,36,2.40
3,GVOFK,26,59,2.27
4,JSUUP,1,2,2.00
7,RT9Z6,1,2,2.00
2,EUZ/Q,1,1,1.00


First, I got which unique user_id have visited at least two different countries(df_id). Second, I selected columns, 'user_id' and 'site_id' from the full data, and dropped duplicates. From this second dataset, I chose the user_id who visited at least two different countries by using the first dataset(df_id), then counted the number of unique user_id by the site_id(A). In order to find B, I selected columns, 'user_id', 'site_id' from the full data, and dropped duplicates. Then, I found the number of unique users that visited each site_id(B). I merged the two datasets(df_a, df_b) on 'site_id' and calculated the ratio, B/A as shown above. 
The top 3 sites are "QGO3G","5NPAU", and "N0OTG", and their corresponding ratios are 4.47, 4.25, and 3.68, respectively.